In [1]:
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy
from itertools import count

def action_num (game_object, action):
    for i, act in zip(count(), game_object.ACTION_DICT.keys()):
        if action == act :
            return i

def prepare_data (game_object, q_table):
    training_table = dict()
    output_template = [0 for _ in range(len(game_object.ACTION_DICT))]
    for state_action , value in q_table :
        (state , action) = state_action
        state = state + game_object.SOLVED_STATE
        inner = training_table[state] or output_template.copy()
        inner[action_num(action)] = value
        training_table[state] = inner
    return list(training_table.keys()) , list(training_table.values())

def import_q_table (filename="floppy_q_table.txt"):
    q_table = dict()
    with open(filename, "r") as file:
        for line in file.readlines():
            if line == "Q_table = {\n" or line == "}":
                continue
            state_action, value = line[:-2].split(":")
            state = tuple([int(x.strip(" ")) for x in state_action[2:-5].split(",")])
            action = int(state_action[-2])
            q_table[(state, action)] = float(value)
    return q_table

In [2]:
def initilialize_nn (game_object):
    actions = game_object.ACTION_DICT
    solved_state = game_object.SOLVED_STATE
    input_size = len(solved_state)*2
    output_size = len(actions)
    model = keras.Sequential()
    model.add(keras.Input(shape=(input_size)))
    model.add(keras.Dense((input_size+output_size)/2))
    model.add(keras.Dense(output_size))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [3]:
def train_nn (model, samples, labels, epochs=100, batch_size=30):
    return model.fit(samples, labels, epochs=epochs, batch_size=batch_size,
                     use_multiprocessing=True, verbose=False)

In [1]:
from puzzle_class import Twisty_Puzzle

puzzle = Twisty_Puzzle()
puzzle.load_puzzle("floppy_cube")

q_table = import_q_table()
inputs, outputs = prepare_data(puzzle, q_table)
model = initialize_nn(puzzle)
history = train_nn(model, inputs, outputs)

<IPython.core.display.Javascript object>

NameError: name 'TwistyPuzzle' is not defined